In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.append('../qiskit_device_benchmarking/')

In [ ]:
from qiskit_ibm_provider import IBMProvider
# from qiskit import IBMQ
# IBMQ.load_account()

provider =IBMProvider()
backend = provider.get_backend('ibm_torino')
backend.status()

In [ ]:
from qiskit_aer import AerSimulator

backend_sim = AerSimulator.from_backend(backend)
backend = backend_sim

In [ ]:
# from bell_experiment import get_bell_layers
from bench_code.bell.partition import partition_qubit_pairs,partition_qubits
from bench_code.bell.bell_experiment import BellExperiment
qubits_nns = partition_qubits(backend,distance=2)
qubits_nnns = partition_qubits(backend,distance=3)
layered_coupling_map =  partition_qubit_pairs(backend,distance=2)

In [ ]:
from qiskit_experiments.library import T2Ramsey,Tphi,T1,StandardRB

# Time intervals to wait before measurement for t1 and t2
delays_t1 = np.arange(1e-6, 300e-6, 15e-6)
delays_t1 = [0] + list(np.logspace(np.log10(10e-9),np.log10(200e-6),41))

delays_t2 = [0] + np.arange(1e-6, 300e-6, 15e-6).tolist()
num_periods = 5
max_T = delays_t2[-1]
osc_freq = num_periods/(max_T)

In [ ]:
from qiskit_experiments.framework import ParallelExperiment, BatchExperiment

exp_batches = []
for qubits in qubits_nns:
    coh_exps = ParallelExperiment([
        Tphi((int(qubit),), delays_t1=delays_t1,delays_t2=delays_t2,t2type='ramsey',osc_freq=osc_freq,backend=backend)
        for qubit in np.array(qubits).flatten()
    ], flatten_results=True)
    exp_batches.append(coh_exps)


In [ ]:
from bell_experiment import BellExperiment
bell_exp = BellExperiment(layered_coupling_map,backend=backend)

In [ ]:
exp_batches.append(bell_exp)

In [ ]:
batch_exp = BatchExperiment(exp_batches,backend=backend,flatten_results=True)

In [ ]:
batch_exp_data = batch_exp.run()

In [ ]:
batch_exp_data.status()

In [ ]:
i=1
print(batch_exp_data.figure(i).metadata)
batch_exp_data.figure(i).figure